In [59]:
from epformatline.worker import translate

In [60]:
from pathlib import Path
import re
import subprocess
from typing import Optional, List

import difflib as dl


In [61]:
ROOT_DIR = Path('../../').resolve()

In [62]:
EP_SRC = ROOT_DIR / 'src/EnergyPlus'

# Regex-fu

In [63]:
FUNCTION_NAMES = [
    'ShowFatalError',
    'ShowSevereError',
    'ShowSevereMessage',
    'ShowContinueError',
    'ShowContinueErrorTimeStamp',
    'ShowMessage',
    'ShowWarningError',
    'ShowWarningMessage',
    'ShowErrorMessage',
]

RECURRING_FUNCTION_NAMES = [
    'ShowRecurringSevereErrorAtEnd',
    'ShowRecurringWarningErrorAtEnd',
    'ShowRecurringContinueErrorAtEnd',
    'StoreRecurringErrorMessage',
]

In [64]:
func_pattern = rf"(?P<function>{'|'.join(FUNCTION_NAMES)})"
func_pattern

'(?P<function>ShowFatalError|ShowSevereError|ShowSevereMessage|ShowContinueError|ShowContinueErrorTimeStamp|ShowMessage|ShowWarningError|ShowWarningMessage|ShowErrorMessage)'

In [65]:
RE_MESSAGE = re.compile(
    func_pattern +
    r'\s*?\(\s*?(?P<state>\w+)\s*?,\s+(?P<input>.*?)\s*?\);',
    flags=re.DOTALL | re.MULTILINE
)

In [66]:
RE_MESSAGE

re.compile(r'(?P<function>ShowFatalError|ShowSevereError|ShowSevereMessage|ShowContinueError|ShowContinueErrorTimeStamp|ShowMessage|ShowWarningError|ShowWarningMessage|ShowErrorMessage)\s*?\(\s*?(?P<state>\w+)\s*?,\s+(?P<input>.*?)\s*?\);',
re.MULTILINE|re.DOTALL|re.UNICODE)

In [67]:
def replace_fun(matchobj):
    d = matchobj.groupdict()
    input_str = d['input'].strip()
    if input_str.startswith('format') or input_str.startswith('fmt::format') or not '+' in input_str:
        return matchobj.group(0) # No-op
    return f"{d['function']}({d['state']}, {translate(input_str)});"

# Test content

In [68]:
test_content = r"""
        //   Warn user if curve output goes negative
        if (TotCoolCapTempModFac < 0.0) {
            if (!state.dataGlobal->WarmupFlag && NumTUInCoolingMode > 0) {
                if (vrf.CoolCapFTErrorIndex == 0) {
                    ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");
                    ShowContinueError(
                        state,
                        format(" Cooling Capacity Modifier curve (function of temperature) output is negative ({:.3T}).", TotCoolCapTempModFac));
                    ShowContinueError(state,
                                      format(" Negative value occurs using an outdoor air temperature of {:.1T} C and an average indoor air "
                                             "wet-bulb temperature of {:.1T} C.",
                                             CondInletTemp,
                                             InletAirWetBulbC));
                    ShowContinueErrorTimeStamp(state, " Resetting curve output to zero and continuing simulation.");
                }
                ShowRecurringWarningErrorAtEnd(
                    state,
                    format("{} \"{}\": Cooling Capacity Modifier curve (function of temperature) output is negative warning continues...",
                           PlantEquipTypeNames[static_cast<int>(PlantEquipmentType::HeatPumpVRF)],
                           vrf.Name),
                    vrf.CoolCapFTErrorIndex,
                    TotCoolCapTempModFac,
                    TotCoolCapTempModFac);
                TotCoolCapTempModFac = 0.0;
            }
        }"""

In [69]:
print(RE_MESSAGE.sub(replace_fun, test_content))


        //   Warn user if curve output goes negative
        if (TotCoolCapTempModFac < 0.0) {
            if (!state.dataGlobal->WarmupFlag && NumTUInCoolingMode > 0) {
                if (vrf.CoolCapFTErrorIndex == 0) {
                    ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));
                    ShowContinueError(
                        state,
                        format(" Cooling Capacity Modifier curve (function of temperature) output is negative ({:.3T}).", TotCoolCapTempModFac));
                    ShowContinueError(state,
                                      format(" Negative value occurs using an outdoor air temperature of {:.1T} C and an average indoor air "
                                             "wet-bulb temperature of {:.1T} C.",
                                             CondInletTemp,
                                             InletAirWetBulbC));
                    ShowContinueErrorTimeStamp(state, " Resettin

# Try on a single file

In [70]:
def clang_format(fpath: Path):
    subprocess.check_call(['clang-format-11', '--style=file', '-i', str(fpath)])

In [71]:
def strip(lines):
    return [x.strip() for x in lines]

class ReportInfo:
    def __init__(self, file: Path, lines: [str], fixed_lines: [str]):
        self.file = file
        self.lines = lines
        self.fixed_lines = fixed_lines
        self.num_lines_changed = len(set(fixed_lines) - set(lines))
    
    def show_diff(self, style_output: bool = False, dedent: bool = True, use_generator: bool = False, width: int = None):
        
        if dedent:
            diffs = dl.unified_diff(strip(self.lines), strip(self.fixed_lines), n=0, lineterm="")
        else:
            diffs = dl.unified_diff(self.lines, self.fixed_lines, n=0, lineterm="")

        if width is None:
            width = 115
        
        if style_output:
            from rich.console import Console
            from rich.syntax import Syntax
            console = Console(width=width)
            if use_generator:
                for diff in diffs:
                    syntax = Syntax(diff, "Diff", theme="", line_numbers=False, word_wrap=True)
                    console.print(syntax)
            else:
                all_diff = "\n".join(list(diffs))
                syntax = Syntax(all_diff, "Diff", theme="", line_numbers=False, word_wrap=True)
                console.print(syntax)
        else:
            if use_generator:
                for diff in diffs:
                    print(diff)
            else:
                all_diff = "\n".join(list(diffs))
                print(all_diff)
        
    def __repr__(self):
        return f"{self.file.relative_to(ROOT_DIR)}: {self.num_lines_changed} changes"

def fix_format_in_file(file: Path, inplace: bool = True, reformat: bool = True) -> Optional[ReportInfo]:
    with open(file, 'r') as f:
        content = f.read()
    if not(any(x in content for x in FUNCTION_NAMES)):
        # print(f"Skipping {file}, no print function in there")
        return
    fixed_content = RE_MESSAGE.sub(replace_fun, content)
    fixed_lines = fixed_content.splitlines()
    if inplace:
        with open(file, 'w') as f:
            f.write(fixed_content)
            if reformat:
                clang_format(file)
        
    lines = content.splitlines()
    fixed_lines = fixed_content.splitlines()
    return ReportInfo(file=file, lines=lines, fixed_lines=fixed_lines)

In [72]:
fpath = EP_SRC / 'HVACVariableRefrigerantFlow.cc'
r = fix_format_in_file(file=fpath)

In [73]:
r

src/EnergyPlus/HVACVariableRefrigerantFlow.cc: 200 changes

In [74]:
width = None
r.show_diff(style_output=True, dedent=True, width=width)

---                                                                                                                
+++                                                                                                                
@@ -170 +170 @@                                                                                                    
-ShowFatalError(state, "SimulateVRF: VRF Terminal Unit not found=" + std::string{CompName});                       
+ShowFatalError(state, format("SimulateVRF: VRF Terminal Unit not found={}", CompName));                           
@@ -300 +300 @@                                                                                                    
-ShowFatalError(state, "LocalVRFCondenserFactory: Error getting inputs for object named: " + objectName); //       
LCOV_EXCL_LINE                                                                                                     
+ShowFatalError(state, format("LocalVRFCondenserFactory: Error getting inputs for object named: {}", objectName)); 
// LCOV_EXCL_LINE                                                                                                  
@@ -341 +341 @@                                                                                                    
-ShowFatalError(state, "SimVRFCondenserPlant:: Invalid loop connection " + std::string(cVRFTypes(VRF_HeatPump)));  
+ShowFatalError(state, format("SimVRFCondenserPlant:: Invalid loop connection {}", cVRFTypes(VRF_HeatPump)));      
@@ -605 +605 @@                                                                                                    
-ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");                        
+ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));                                
@@ -632 +632 @@                                                                                                    
-ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");                        
+ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));                                
@@ -668 +668 @@                                                                                                    
-ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");                        
+ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));                                
@@ -694 +694 @@                                                                                                    
-ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");                        
+ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));                                
@@ -776 +776 @@                                                                                                    
-ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");                        
+ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));                                
@@ -818 +818 @@                                                                                                    
-ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");                        
+ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));                                
@@ -891 +891 @@                                                                                                    
-ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");                        
+ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));                                
@@ -1366,3 +1366 @@                                     

# Change width of notebook cells for viewing diffs

In [75]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

notebook_width = '90%'
if notebook_width == '100%':
    width = 184
elif notebook_width == '90%':
    width = 164
else:
    width = 115

In [76]:
r.show_diff(style_output=True, dedent=True, width=width)

---                                                                                                                                                                 
+++                                                                                                                                                                 
@@ -170 +170 @@                                                                                                                                                     
-ShowFatalError(state, "SimulateVRF: VRF Terminal Unit not found=" + std::string{CompName});                                                                        
+ShowFatalError(state, format("SimulateVRF: VRF Terminal Unit not found={}", CompName));                                                                            
@@ -300 +300 @@                                                                                                                                                     
-ShowFatalError(state, "LocalVRFCondenserFactory: Error getting inputs for object named: " + objectName); // LCOV_EXCL_LINE                                         
+ShowFatalError(state, format("LocalVRFCondenserFactory: Error getting inputs for object named: {}", objectName)); // LCOV_EXCL_LINE                                
@@ -341 +341 @@                                                                                                                                                     
-ShowFatalError(state, "SimVRFCondenserPlant:: Invalid loop connection " + std::string(cVRFTypes(VRF_HeatPump)));                                                   
+ShowFatalError(state, format("SimVRFCondenserPlant:: Invalid loop connection {}", cVRFTypes(VRF_HeatPump)));                                                       
@@ -605 +605 @@                                                                                                                                                     
-ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");                                                                         
+ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));                                                                                 
@@ -632 +632 @@                                                                                                                                                     
-ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");                                                                         
+ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));                                                                                 
@@ -668 +668 @@                                                                                                                                                     
-ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");                                                                         
+ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));                                                                                 
@@ -694 +694 @@                                                                                                                                                     
-ShowSevereMessage(state, std::string(cVRFTypes(VRF_HeatPump)) + " \"" + vrf.Name + "\":");                                                                         
+ShowSevereMessage(state, format("{} \"{}\":", cVRFTypes(VRF_HeatPump), vrf.Name));                                                                                 
@@ -776 +776 @@                                                                                                                                                     
-ShowSevereMessage(state, std::string(cV

## Run all files

In [94]:
clang_format(EP_SRC / 'VentilatedSlab.cc')

In [77]:
EP_SRC

PosixPath('/Users/julien/Software/Others/EnergyPlus2/src/EnergyPlus')

In [79]:
%%time
all_reports = []
for fpath in EP_SRC.glob('*.cc'):
    if not fpath.is_file():
        continue
    if not fpath.suffix in ['.cc', '.hh']:
        print(f"Skipping extension for {fpath}")
        continue
    r = fix_format_in_file(file=fpath)
    if r:
        all_reports.append(r)

CPU times: user 691 ms, sys: 501 ms, total: 1.19 s
Wall time: 13.9 s
